In [ ]:
import pandas as pd
import os
import tmpo

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
token = "b371402dc767cc83e41bc294b63f9586"
house = {
    "cellar": {
        "electricity": "fed676021dacaaf6a12a8dda7685be34"
    }
}

## Init session, add sensors and sync

In [ ]:
s = tmpo.Session(path=os.getcwd())
for room in house:
    for sensor in house[room]:
        s.add(house[room][sensor], token)

In [ ]:
s.debug = True

In [ ]:
s.sync()

## Get info about sensor

In [ ]:
sensor = house[room]["electricity"]

type, data_type, subtype = s.get_types(sensor)
unit = s.get_unit(sensor)

print(type, data_type, subtype, unit)

## Get raw data

In [ ]:
s.debug = False

In [ ]:
start = pd.Timestamp('20160101')
end = pd.Timestamp('20160501')

In [ ]:
df = s.dataframe(sids=[house[room]["electricity"]], head=start, tail=end)

In [ ]:
room = "cellar"
plt.figure(figsize=(16,5))
ax1=plt.subplot()
ax1.grid()
ax1.set_ylabel(u'Wh')
plt.title(room + " electricity")
df.plot(ax=ax1)

## Get resampled data

The difference between raw and resampled data is that raw data has data with an uneven time axis. This makes it difficult to calculate power values or flow rates.

In [ ]:
df = s.get_data(sids=[house[room]["electricity"]], head=start, tail=end, resolution='1h')

As resolution you can pick from these resolutions, and you can precede them with a number:
* B   business day frequency
* C   custom business day frequency (experimental)
* D   calendar day frequency
* W   weekly frequency
* M   month end frequency
* BM  business month end frequency
* MS  month start frequency
* BMS business month start frequency
* Q   quarter end frequency
* BQ  business quarter endfrequency
* QS  quarter start frequency
* BQS business quarter start frequency
* A   year end frequency
* BA  business year end frequency
* AS  year start frequency
* BAS business year start frequency
* H   hourly frequency
* T   minutely frequency
* S   secondly frequency
* L   milliseconds
* U   microseconds

In [ ]:
room = "cellar"
plt.figure(figsize=(16,5))
ax1=plt.subplot()
ax1.grid()
ax1.set_ylabel(u'Wh')
plt.title(room + " electricity")
df.plot(ax=ax1)

Now you can easily calculate power by taking the difference between values.

In [ ]:
P = df.diff()

In [ ]:
room = "cellar"
plt.figure(figsize=(16,5))
ax1=plt.subplot()
ax1.grid()
ax1.set_ylabel(u'W')
plt.title(room + " electricity")
P.plot(ax=ax1)

Mind your dimensions! Since we had Wh values resampled to hourly basis, the `diff` gets us Wh/h, or W! If you choose another resolution you need to bring a factor into account.

## Export to CSV, Excel...

In [ ]:
P.to_csv('tmpo_example.csv')

In [ ]:
P.to_excel('tmpo_example.xlsx')

Pandas has many other export methods, see https://pandas.pydata.org/pandas-docs/stable/io.html